In [585]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import gmaps
import gmaps.geojson_geometries
import os
import json
import requests
import pprint as pp

import csv



# Import API key
from config import g_key

In [2]:
# File to Load
NYC1_data_to_load = "resources/NYC_Listings1.csv"
NYC2_data_to_load = "resources/NYC_Listings2.csv"
NYC3_data_to_load = "resources/NYC_Listings3.csv"

# Read School and Student Data File and store into Pandas Data Frames
NYC1_data = pd.read_csv(NYC1_data_to_load)
NYC2_data = pd.read_csv(NYC2_data_to_load)
NYC3_data = pd.read_csv(NYC3_data_to_load)

/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mikeyesteban/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Create a full NYC DF
NYC_data = pd.concat([NYC1_data, NYC2_data, NYC3_data], ignore_index=True)
NYC_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.019080e+13,8/7/19,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2.0,1.0,0.0,1.0,0.39
1,3647,https://www.airbnb.com/rooms/3647,2.019080e+13,8/6/19,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,0.0,1.0,0.0,NaN
2,3831,https://www.airbnb.com/rooms/3831,2.019080e+13,8/6/19,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1.0,1.0,0.0,0.0,4.64
3,5022,https://www.airbnb.com/rooms/5022,2.019080e+13,8/6/19,Entire Apt: Spacious Studio/Loft by central park,NaN,Loft apartment with high ceiling and wood floo...,Loft apartment with high ceiling and wood floo...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.10
4,5099,https://www.airbnb.com/rooms/5099,2.019080e+13,8/6/19,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment is true New York ...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment is true New York ...,none,My neighborhood in Midtown East is called Murr...,...,f,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.60


In [4]:
# Review columns
column_names = list(NYC_data.columns)
column_names[:5]

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']

In [5]:
# Select Columns for clean DataFrame
Clean_NYC = NYC_data[['id','name','neighbourhood',
                      'latitude','longitude','property_type',
                      'room_type','accommodates','amenities',
                      'price','availability_365','number_of_reviews',
                     'number_of_reviews_ltm','review_scores_rating',
                      'review_scores_location','reviews_per_month']]
# Drop null values
Clean_NYC = Clean_NYC.dropna()

# Change price value to float
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip('$'))
#Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.lstrip(','))
Clean_NYC['price'] = Clean_NYC['price'].map(lambda x: x.replace(',',''))
Clean_NYC['price'] = Clean_NYC['price'].astype('float')

# Preview DataFrame
Clean_NYC

,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,Apartment,Entire home/apt,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",225.0,288.0,46.0,12.0,95.0,10.0,0.39
2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,Guest suite,Entire home/apt,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",89.0,212.0,274.0,70.0,90.0,10.0,4.64
3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,Apartment,Entire home/apt,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",80.0,0.0,9.0,2.0,93.0,9.0,0.10
4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",200.0,127.0,75.0,10.0,89.0,9.0,0.60
5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,Apartment,Private room,2.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",60.0,0.0,49.0,0.0,90.0,9.0,0.39
6,5178,Large Furnished Room Near B'way,Manhattan,40.76489,-73.98493,Apartment,Private room,2.0,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",79.0,239.0,434.0,44.0,84.0,10.0,3.48
7,5203,Cozy Clean Guest Room - Family Apt,Upper West Side,40.80178,-73.96723,Apartment,Private room,1.0,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",79.0,0.0,118.0,0.0,98.0,10.0,0.98
8,5222,Best Hideaway,Manhattan,40.72764,-73.97949,Apartment,Entire home/apt,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",116.0,322.0,94.0,0.0,95.0,10.0,0.74
9,5238,Cute & Cozy Lower East Side 1 bdrm,Lower East Side,40.71344,-73.99037,Apartment,Entire home/apt,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",150.0,0.0,161.0,13.0,94.0,9.0,1.33
10,5295,Beautiful 1br on Upper West Side,Manhattan,40.80316,-73.96545,Apartment,Entire home/apt,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,Door...",135.0,1.0,53.0,14.0,93.0,9.0,0.42


In [112]:
# GroupBY Neighborhood
neighborhoods_ratings = Clean_NYC.groupby('neighbourhood')['review_scores_location'].mean()
neighborhoods_ratings.sort_values(ascending=False, inplace=True)
neighborhoods_ratings.head()
# USE GOOGLE PLACES API TO CREATE A MAP THATS SPLIT

neighbourhood
Times Square/Theatre District    10.0
Grant City                       10.0
City Island                      10.0
Oakwood                          10.0
Woodlawn                         10.0
Name: review_scores_location, dtype: float64

In [37]:
neighborhoods_list = list(neighborhoods_ratings.index)
neighborhoods_list[:10]
len(neighborhoods_list)

190

In [9]:
# Create a Dictionary with neighbourhood DataFrames
d = {}
for city in neighborhoods_list:
    checker = (Clean_NYC['neighbourhood'] == city)
    city_df = Clean_NYC.where(checker)
    city_df = city_df.dropna()
    d[city] = city_df


In [565]:
final_neighborhood_list = list(d.keys())
final_neighborhood_list.sort()
final_neighborhood_list

['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brownsville',
 'Bushwick South',
 'Canarsie',
 'Carroll Gardens-Columbia Street-Red Hook',
 'Castleton Corners',
 'Chinatown',
 'Claremont-Bathgate',
 'Clinton',
 'Clinton Hill',
 'College Point',
 'Corona',
 'Crotona Park East',
 'Crown Heights North',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Dyker Heights',
 'East Concourse-Concourse Village',
 'East Elmhurst',
 'East Flatbush-Farragut',
 'East Harlem South',
 'East New York',
 'East Village',
 'Eastchester-Edenwald-Baychester',
 'Elmhurst',
 'Flatbush',
 'Flatlands',
 'Flushing',
 'Forest Hills',
 'Fort Greene',
 'Fresh Meadows-Utopia',
 'Georgetown-Marine Park-Bergen Bea

In [608]:
# SAVING MY EDITED DICTIONARY 
def saver(d):
    for key, val in d.items():
        val.to_csv("csvfiles/data_{}.csv".format(str(key)))

    with open("keys.txt", "w") as f: #saving keys to file
        f.write(str(list(d.keys())))



In [611]:
saver(d)

FileNotFoundError: [Errno 2] No such file or directory: 'data_Times Square/Theatre District.csv'

In [609]:
# LOADING MY EDITED DICTIONARY
def loader():
    """Reading data from keys"""
    with open("keys.txt", "r") as f:
        keys = eval(f.read())

    d = {}    
    for key in keys:
        d[key] = pd.read_csv("data_{}.csv".format(str(key)))

    return d

### CHANGING NAMES AND UPDATING DICTIONARY

In [115]:
# Function to Change Names
def change_name(self,key,new_key):
    value = self[key]
    self.pop(key)
    self[new_key] = value

In [118]:
# Functino to Update Dictionary
def title_name_change(self,old_key, new_key):
    change_name(self,old_key,new_key)
    dataframe = self[new_key]
    dataframe['new neighborhood'] = new_key

In [582]:
title_name_change(d,"Morris Park","Van Nest-Morris Park-Westchester Square")
d["Van Nest-Morris Park-Westchester Square"].head()



,id,name,neighbourhood,latitude,longitude,property_type,room_type,accommodates,amenities,price,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_location,reviews_per_month,new neighborhood
20778,16777456.0,Large Studio New York-Bronx comfortable,Morris Park,40.84906,-73.85418,Condominium,Private room,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,Es...",73.0,145.0,14.0,6.0,97.0,10.0,0.59,Van Nest-Morris Park-Westchester Square
23983,19661733.0,Private room at Stella's place,Morris Park,40.85152,-73.86121,House,Private room,2.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Free stre...",39.0,69.0,61.0,22.0,99.0,10.0,2.45,Van Nest-Morris Park-Westchester Square
33441,27016931.0,Bronx Apartment-> BRAND NEW BUILDING-BUILT IN ...,Morris Park,40.85322,-73.85144,House,Private room,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",55.0,59.0,6.0,6.0,97.0,10.0,0.51,Van Nest-Morris Park-Westchester Square
34806,28206135.0,Bronx ROOM #2 NYC-BEST DEAL,Morris Park,40.85409,-73.85032,House,Private room,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",50.0,51.0,9.0,9.0,98.0,9.0,0.89,Van Nest-Morris Park-Westchester Square
34807,28206177.0,Bronx Brand New Room #3,Morris Park,40.85361,-73.85018,House,Private room,3.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",81.0,328.0,5.0,5.0,100.0,10.0,0.45,Van Nest-Morris Park-Westchester Square


In [583]:
# ADDING LISTS TOGETHER
d["Van Nest-Morris Park-Westchester Square"] = d["Van Nest-Morris Park-Westchester Square"].append(d["Van Nest"], ignore_index=False)



In [584]:
# adding new neighborhood name
d["Van Nest-Morris Park-Westchester Square"]['new neighborhood'] = "Van Nest-Morris Park-Westchester Square"



In [570]:
#d['SoHo-TriBeCa-Civic Center-Little Italy']
#list(d.keys())
#d['DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill']
len(d["Georgetown-Marine Park-Bergen Beach-Mill Basin"])

2

In [566]:
final_neighborhood_list


['Allerton-Pelham Gardens',
 "Annadale-Huguenot-Prince's Bay-Eltingville",
 'Astoria',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Belmont',
 'Bensonhurst West',
 'Borough Park',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn',
 'Brooklyn Heights-Cobble Hill',
 'Brownsville',
 'Bushwick South',
 'Canarsie',
 'Carroll Gardens-Columbia Street-Red Hook',
 'Castleton Corners',
 'Chinatown',
 'Claremont-Bathgate',
 'Clinton',
 'Clinton Hill',
 'College Point',
 'Corona',
 'Crotona Park East',
 'Crown Heights North',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Dyker Heights',
 'East Concourse-Concourse Village',
 'East Elmhurst',
 'East Flatbush-Farragut',
 'East Harlem South',
 'East New York',
 'East Village',
 'Eastchester-Edenwald-Baychester',
 'Elmhurst',
 'Flatbush',
 'Flatlands',
 'Flushing',
 'Forest Hills',
 'Fort Greene',
 'Fresh Meadows-Utopia',
 'Georgetown-Marine Park-Bergen Bea

In [580]:
d.pop("University Heights")

KeyError: 'University Heights'

In [150]:
new_neighborhood_json_list = []

with open('resources/nyc-neighborhoods.geo.json') as f:
    geometry = json.load(f)
    
#     for key,value in geometry.items():
#         print(geometry['features'][0])

    for row in geometry['features']:
        city = row['properties']['name']
        if city not in new_neighborhood_json_list:
            new_neighborhood_json_list.append(city)

len(new_neighborhood_json_list)

188

In [581]:
for row in new_neighborhood_json_list:
    if "Morris Park" in row:
        print(row)

Van Nest-Morris Park-Westchester Square


In [567]:
# DATA MUNGING TO FIX CITY MAP
final_neighborhood_list
new_neighborhood_json_list
new_missing_cities = []
new_extra_json_cities = []


for city in final_neighborhood_list:
    if city not in new_neighborhood_json_list:
        new_missing_cities.append(city)
        
for city in new_neighborhood_json_list:
    if city not in final_neighborhood_list:
        new_extra_json_cities.append(city)

print('MISSING CITIES TO ADD', len(new_missing_cities))
print('EXTRA CITIES IN GRAPH', len(new_extra_json_cities))
new_missing_cities



#ALPHABETIZE Lists
new_neighborhood_json_list.sort()
new_missing_cities.sort()
len(new_neighborhood_json_list)


checker_df = pd.DataFrame({'missing cities': pd.Series(new_missing_cities), 
              'json list': pd.Series(new_neighborhood_json_list),
             'extra json to use': pd.Series(new_extra_json_cities)})
checker_df

#'Williamsburg' in new_neighborhood_json_list

MISSING CITIES TO ADD 41
EXTRA CITIES IN GRAPH 84


,missing cities,json list,extra json to use
0,Brooklyn,Allerton-Pelham Gardens,Arden Heights
1,Castleton Corners,Annadale-Huguenot-Prince's Bay-Eltingville,Auburndale
2,Manhattan,Arden Heights,Baisley Park
3,Midtown East,Astoria,Bellerose
4,Mill Basin,Auburndale,Bensonhurst East
5,Morris Heights,Baisley Park,Breezy Point-Belle Harbor-Rockaway Park-Broad ...
6,Morris Park,Bath Beach,Briarwood-Jamaica Hills
7,Morrisania,Battery Park City-Lower Manhattan,Bushwick North
8,New Springville,Bay Ridge,Cambria Heights
9,Noho,Bayside-Bayside Hills,Central Harlem North-Polo Grounds


# THE NEW NEIGHBORHOOD COLUMN IS THE SAME AS JSON NAMES

In [ ]:
# # Configure gmaps with API key
# gmaps.configure(api_key=g_key)
# # Create a Gmaps figure
# fig = gmaps.figure()

# locations = Clean_NYC[['latitude','longitude']]
# weights = Clean_NYC['review_scores_location']
# # Create the Heat Layer Object 
# heat_layer = gmaps.heatmap_layer(locations=locations, weights=weights, dissipating=True)
# heat_layer.point_radius = 4
# heat_layer.gradient = [
#     'gray',
#     'black',
#     'blue'
# ]
# # Add Heat Layer to Figure
# fig.add_layer(heat_layer)

# fig

In [ ]:
# new_york_coordinates = (40.75, -74.00)
# #gmaps.figure(center=new_york_coordinates, zoom_level=12)

# countries_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

# fig = gmaps.figure(center=new_york_coordinates, zoom_level=12)
# gini_layer = gmaps.geojson_layer(countries_geojson)
# fig.add_layer(gini_layer)
# #fig
